# 🛣️ AI Road Quality Monitoring — YOLOv8 Training on Google Colab

**Runtime → Change runtime type → T4 GPU (FREE)**

This notebook trains YOLOv8 on your road damage dataset and downloads the best weights.

---
**Steps:**
1. Check GPU
2. Install dependencies
3. Download dataset from Roboflow
4. Train YOLOv8m
5. Evaluate (mAP, confusion matrix)
6. Download `best.pt` → use in your local dashboard

In [ ]:
# ── CELL 1: Check GPU ────────────────────────────────────────────────
!nvidia-smi
import torch
print(f'\nPyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# ── CELL 2: Install YOLOv8 ───────────────────────────────────────────
!pip install ultralytics roboflow -q
from ultralytics import YOLO
print('✅ ultralytics installed')

In [ ]:
# ── CELL 3: Download Dataset from Roboflow ───────────────────────────
# Get your FREE API key at https://roboflow.com → Settings → API Keys
# Then replace YOUR_API_KEY_HERE below

from roboflow import Roboflow

API_KEY = 'YOUR_API_KEY_HERE'   # <─── PASTE YOUR KEY HERE

rf      = Roboflow(api_key=API_KEY)

# Option A: Pothole detection dataset (public, no account needed for download)
project = rf.workspace('university-sbxzs').project('pothole-detection-a73yl')
version = project.version(1)
dataset = version.download('yolov8')

print(f'\n✅ Dataset downloaded to: {dataset.location}')
print(f'   Classes: {dataset.classes}')
DATASET_YAML = dataset.location + '/data.yaml'
print(f'   Config:  {DATASET_YAML}')

In [ ]:
# ── CELL 3B (ALTERNATIVE): Use RDD2022 or custom dataset ─────────────
# If you uploaded your own dataset to Colab, set the path here:
#
# from google.colab import files
# uploaded = files.upload()        # upload your dataset zip
# !unzip dataset.zip -d /content/dataset
#
# DATASET_YAML = '/content/dataset/data.yaml'

# Or mount Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# DATASET_YAML = '/content/drive/MyDrive/road_dataset/data.yaml'
print('Use Cell 3 (Roboflow) or uncomment one of the alternatives above.')

In [ ]:
# ── CELL 4: Train YOLOv8 ─────────────────────────────────────────────
# Expected training time on T4 GPU:
#   yolov8n  ~20 min for 100 epochs
#   yolov8m  ~60 min for 100 epochs  ← recommended
#   yolov8l  ~120 min for 100 epochs

from ultralytics import YOLO

# Load pretrained model (auto-downloads ~50MB weights)
model = YOLO('yolov8m.pt')

# ── Training ──────────────────────────────────────────────────────────
results = model.train(
    data       = DATASET_YAML,
    epochs     = 100,
    imgsz      = 640,
    batch      = 32,          # T4 has 16GB VRAM, batch=32 is fine
    device     = 0,           # GPU 0
    patience   = 20,          # Early stopping
    optimizer  = 'AdamW',
    lr0        = 0.001,
    # Augmentation
    mosaic     = 1.0,
    fliplr     = 0.5,
    degrees    = 5.0,
    hsv_v      = 0.4,
    # Output
    project    = '/content/runs',
    name       = 'road_damage',
    plots      = True,
    save       = True,
    verbose    = True,
)

print(f'\n✅ Training complete!')
print(f'   Best mAP50: {results.results_dict.get("metrics/mAP50(B)", 0):.4f}')
print(f'   Results:    {results.save_dir}')

In [ ]:
# ── CELL 5: View Training Plots ───────────────────────────────────────
from IPython.display import Image, display
import glob
from pathlib import Path

run_dir = Path(results.save_dir)

print('=== Training Results ===')
display(Image(str(run_dir / 'results.png'), width=900))

print('\n=== Confusion Matrix ===')
display(Image(str(run_dir / 'confusion_matrix.png'), width=700))

print('\n=== Precision-Recall Curve ===')
display(Image(str(run_dir / 'PR_curve.png'), width=700))

In [ ]:
# ── CELL 6: Evaluate on Test Set ─────────────────────────────────────
best_model = YOLO(str(run_dir / 'weights' / 'best.pt'))

metrics = best_model.val(
    data   = DATASET_YAML,
    split  = 'test',
    imgsz  = 640,
    conf   = 0.25,
    iou    = 0.50,
    plots  = True,
    verbose= True,
)

print(f'\n{"="*50}')
print(f'  TEST SET RESULTS')
print(f'{"="*50}')
print(f'  mAP50:     {metrics.box.map50:.4f}')
print(f'  mAP50-95:  {metrics.box.map:.4f}')
print(f'  Precision: {metrics.box.mp:.4f}')
print(f'  Recall:    {metrics.box.mr:.4f}')
f1 = 2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-10)
print(f'  F1 Score:  {f1:.4f}')

In [ ]:
# ── CELL 7: Run Inference on Sample Images ────────────────────────────
import cv2
from IPython.display import Image as IPImage
import matplotlib.pyplot as plt
import numpy as np

# Run on a few test images
test_imgs = list((Path(DATASET_YAML).parent / 'test' / 'images').glob('*.jpg'))[:4]

if test_imgs:
    results_list = best_model.predict(
        source = [str(p) for p in test_imgs],
        conf   = 0.40,
        save   = True,
        project= '/content/predictions',
        name   = 'test_samples',
    )

    fig, axes = plt.subplots(1, min(4, len(test_imgs)), figsize=(20, 5))
    if len(test_imgs) == 1:
        axes = [axes]

    for ax, result in zip(axes, results_list):
        annotated = result.plot()
        ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        ax.axis('off')
        n_dets = len(result.boxes) if result.boxes else 0
        ax.set_title(f'{n_dets} detections', fontsize=10)

    plt.suptitle('YOLOv8 Road Damage Detection — Sample Predictions', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print('No test images found. Check dataset path.')

In [ ]:
# ── CELL 8: Benchmark Inference Speed ────────────────────────────────
import time
import numpy as np

dummy_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

# Warm up
for _ in range(5):
    best_model.predict(dummy_img, verbose=False)

# Benchmark
N     = 100
times = []
for _ in range(N):
    t0 = time.perf_counter()
    best_model.predict(dummy_img, verbose=False)
    times.append(time.perf_counter() - t0)

avg_ms = np.mean(times)  * 1000
fps    = 1 / np.mean(times)
print(f'Average inference: {avg_ms:.1f} ms | FPS: {fps:.1f}')
print(f'Real-time capable: {"YES" if fps > 25 else "NO — try yolov8n or yolov8s"}')

In [ ]:
# ── CELL 9: Export to ONNX (for PC deployment) ───────────────────────
print('Exporting to ONNX format...')
exported = best_model.export(
    format   = 'onnx',
    imgsz    = 640,
    simplify = True,
    optimize = True,
)
print(f'✅ ONNX model: {exported}')

In [ ]:
# ── CELL 10: Download weights to your PC ─────────────────────────────
# This downloads best.pt (PyTorch) to your local machine

from google.colab import files
import shutil, os

best_pt   = str(run_dir / 'weights' / 'best.pt')
onnx_file = str(run_dir / 'weights' / 'best.onnx')

# Bundle weights + results into a zip
zip_name = 'road_damage_model.zip'
shutil.make_archive(
    '/content/road_damage_model', 'zip',
    str(run_dir / 'weights')
)

print('Downloading model weights...')
print('After download, put best.pt in: road_quality_monitor/weights/best.pt')
files.download('/content/road_damage_model.zip')

# Also download training results
shutil.copy(str(run_dir / 'results.png'), '/content/training_results.png')
files.download('/content/training_results.png')

print('\n✅ Download complete!')
print('   1. Extract road_damage_model.zip')
print('   2. Copy best.pt to road_quality_monitor/weights/best.pt')
print('   3. Run: streamlit run 4_dashboard/app.py')

## 📊 Summary of Expected Results

| Metric | Expected Range | Target |
|--------|---------------|--------|
| mAP50 | 0.65 – 0.85 | > 0.75 |
| Precision | 0.70 – 0.90 | > 0.80 |
| Recall | 0.60 – 0.80 | > 0.70 |
| F1 Score | 0.65 – 0.85 | > 0.75 |
| FPS (T4 GPU) | 60–120 | > 25 |

## 🔧 If mAP is below 0.70
1. Add more training images (aim for 500+ per class)
2. Check annotations for accuracy
3. Train for more epochs (`epochs=150`)
4. Try `yolov8l.pt` (larger model)
5. Use `model.tune()` for automatic hyperparameter search